In [1]:
from src.get_quits_codes import get_bpc_code, get_lpc_code
from src.circuit_from_cx_list import memory_experiment_circuit_from_cx_list
import numpy as np
from src.permute_within_each_stabilizer import random_permutation_within_each_stabilizer
from src.permute_single_stabilizer import permute_single_stabilizer_inplace
from collections import Counter

In [2]:
np.random.seed(42)

cx_list, ancilla_type, data_mapping, ancilla_mapping, lz, lx = get_bpc_code(cx_order='theirs')
qubit_to_name = {v:k for k,v in data_mapping.items()}
qubit_to_name.update({v:k for k,v in ancilla_mapping.items()})

In [3]:
# separate the cxs such that they are done one stabilizer at a time
cx_list = sorted(cx_list, key=lambda x: x[1])

In [4]:
random_permutation_within_each_stabilizer(cx_list)

In [5]:
def get_minimal_error():
    circ = memory_experiment_circuit_from_cx_list(
                cx_list=cx_list,
                ancilla_type=ancilla_type,
                data_mapping=data_mapping,
                ancilla_mapping=ancilla_mapping,
                flag_mapping=dict(),  # No flag mapping used here
                logicals=lz,
                logical_type='Z',
                p_cx=0.,
                p_idle=0.,
                p_measurement_error=0.0,
                p_phenomenological_error=0.01,
                hook_errors=hook_errors,
                number_of_cycles=1,
                flag=False
            )
    errors = circ.search_for_undetectable_logical_errors(
        dont_explore_edges_increasing_symptom_degree=False,
        dont_explore_detection_event_sets_with_size_above=6,
        dont_explore_edges_with_degree_above=9999,
        canonicalize_circuit_errors=True,
    )
    # Uncomment to print the circuit
    # for tick in circ.ticks:
    #     print(tick)

    # Uncomment to print the gates in the circuit
#    for gate in circ:

 #       if gate.name == 'M':
  #          print(gate)
    qubits_in_minimal_error = [error.circuit_error_locations[0].instruction_targets.targets_in_range[0].gate_target.value for error in errors]
    hook_ancillas_in_minimal_error = [qubit_to_name[qubit] for qubit in qubits_in_minimal_error if qubit in ancilla_mapping.values()]
    print([qubit_to_name[qubit] for qubit in qubits_in_minimal_error])
    return hook_ancillas_in_minimal_error, len(errors)

In [6]:
stabilizer_weights = Counter(label for _, label in cx_list)
def all_hook_errors_on_ancilla(anc, p=0.02):
    return [(cx_idx,p) for cx_idx in range(1,stabilizer_weights[anc]-2)]

In [ ]:
# start with hook errors on all ancillas, then keep only hook errors on ancillas in minimal error
for _ in range(1000):
    hook_errors = {ancilla: all_hook_errors_on_ancilla(ancilla) for ancilla, a_type in ancilla_type.items() if a_type == 'X'}
    bad_ancillas, distance = get_minimal_error()
    print(bad_ancillas)
    print('full distance: ', distance)
    hook_errors = {ancilla: all_hook_errors_on_ancilla(ancilla) for ancilla in bad_ancillas}
    while True:
        a = np.random.choice(bad_ancillas)
        permute_single_stabilizer_inplace(cx_list, a)
        bad_ancillas, new_distance = get_minimal_error()
        print('new distance: ', new_distance)
        if new_distance > distance:
            break

['X31', 8, 'X25', 'X2', 'X5', 176]
['X31', 'X25', 'X2', 'X5']
full distance:  6
Changed: (44, 'X2') -> (144, 'X2')
Changed: (2, 'X2') -> (44, 'X2')
Changed: (144, 'X2') -> (26, 'X2')
Changed: (26, 'X2') -> (2, 'X2')
['X31', 8, 'X25', 137, 'X5', 176, 44]
new distance:  7
['X13', 'X15', 'X21', 'X14', 164, 31]
['X13', 'X15', 'X21', 'X14']
full distance:  6
Changed: (136, 'X14') -> (14, 'X14')
Changed: (14, 'X14') -> (136, 'X14')
Changed: (41, 'X14') -> (23, 'X14')
Changed: (23, 'X14') -> (141, 'X14')
Changed: (141, 'X14') -> (41, 'X14')
['X13', 'X15', 'X21', 'X14', 164, 31, 41]
new distance:  7
['X16', 'X32', 'X41', 159, 177, 'X25']
['X16', 'X32', 'X41', 'X25']
full distance:  6
Changed: (153, 'X16') -> (1, 'X16')
Changed: (1, 'X16') -> (151, 'X16')
Changed: (158, 'X16') -> (16, 'X16')
Changed: (151, 'X16') -> (158, 'X16')
Changed: (16, 'X16') -> (40, 'X16')
Changed: (40, 'X16') -> (153, 'X16')
[1, 'X32', 153, 'X41', 159, 177, 'X25']
new distance:  7
[9, 'X34', 'X37', 'X12', 'X7', 'X0']
[

In [ ]:
sorted([cx for cx in cx_list if cx[0] in [0, 128, 132, 516, 6, 262, 519, 7, 136, 392, 268, 524]], key=lambda x: x[1])

[(0, 'X0'),
 (262, 'X102'),
 (268, 'X108'),
 (262, 'X118'),
 (268, 'X124'),
 (262, 'X134'),
 (268, 'X140'),
 (392, 'X152'),
 (0, 'X16'),
 (392, 'X168'),
 (392, 'X184'),
 (516, 'X196'),
 (519, 'X199'),
 (524, 'X204'),
 (516, 'X212'),
 (519, 'X215'),
 (6, 'X22'),
 (524, 'X220'),
 (516, 'X228'),
 (7, 'X23'),
 (519, 'X231'),
 (524, 'X236'),
 (0, 'X32'),
 (6, 'X38'),
 (7, 'X39'),
 (128, 'X48'),
 (132, 'X52'),
 (136, 'X56'),
 (6, 'X6'),
 (128, 'X64'),
 (132, 'X68'),
 (7, 'X7'),
 (136, 'X72'),
 (128, 'X80'),
 (132, 'X84'),
 (136, 'X88'),
 (0, 'Z0'),
 (128, 'Z0'),
 (524, 'Z12'),
 (268, 'Z12'),
 (0, 'Z160'),
 (262, 'Z160'),
 (516, 'Z163'),
 (128, 'Z163'),
 (519, 'Z166'),
 (392, 'Z166'),
 (6, 'Z166'),
 (268, 'Z166'),
 (132, 'Z167'),
 (7, 'Z167'),
 (524, 'Z171'),
 (136, 'Z171'),
 (516, 'Z4'),
 (132, 'Z4'),
 (6, 'Z6'),
 (262, 'Z6'),
 (7, 'Z7'),
 (519, 'Z7'),
 (392, 'Z8'),
 (136, 'Z8'),
 (268, 'Z80'),
 (0, 'Z80'),
 (128, 'Z82'),
 (519, 'Z82'),
 (132, 'Z86'),
 (6, 'Z86'),
 (7, 'Z87'),
 (524, 'Z87'),